In [3]:
class Piece:
    """Класс для всех фигур."""

    def __init__(self, color, symbol):
        """
            color - Цвет фигуры.
            symbol - Символ, представляющий фигуру на доске.
        """
        self.color = color
        self.symbol = symbol

    def __str__(self):
        return self.symbol

    def possible_moves(self, board, start_row, start_col):
        """
        Возвращает список возможных ходов для фигуры.
        """
        return []

    def is_valid_move(self, board, start_row, start_col, end_row, end_col):
        """Проверяет, является ли ход допустимым. Так же проверяет, есть ли ход в списке возможных."""
        return (end_row, end_col) in self.possible_moves(board, start_row, start_col)

    def can_capture(self, board, start_row, start_col, end_row, end_col):
        """Проверяет, может ли фигура захватить другую фигуру."""
        if not (0 <= end_row < 8 and 0 <= end_col < 8):
            return False  # Выход за пределы доски

        target_piece = board.board[end_row][end_col]
        if target_piece is None:
            return False  # Нет фигуры для захвата

        return target_piece.color != self.color  # Можно захватить, если цвет цели отличается от цвета текущей фигуры

class Pawn(Piece):
    def __init__(self, color):
        super().__init__(color, '♙' if color == 'white' else '♟')
        self.has_moved = False  # Флаг для отслеживания первого хода
        self.can_promote = False  # Флаг для превращения пешки

    def possible_moves(self, board, start_row, start_col):
        moves = []
        direction = -1 if self.color == 'white' else 1

        # Ход вперед на одну клетку
        new_row = start_row + direction
        if 0 <= new_row < 8 and board.board[new_row][start_col] is None:
            moves.append((new_row, start_col))

            # Ход вперед на две клетки (только для первого хода)
            if not self.has_moved:
                if board.board[new_row + direction][start_col] is None:
                    moves.append((new_row + direction, start_col))

        # Атака по диагонали
        new_col_left = start_col - 1
        new_col_right = start_col + 1

        if 0 <= new_col_left < 8:
            target_piece = board.board[new_row][new_col_left]
            if target_piece is not None and target_piece.color != self.color:
                moves.append((new_row, new_col_left))

        if 0 <= new_col_right < 8:
            target_piece = board.board[new_row][new_col_right]
            if target_piece is not None and target_piece.color != self.color:
                moves.append((new_row, new_col_right))

        # Взятие на проходе
        if board.en_passant_target:
            target_row, target_col = board.en_passant_target
            if start_row == target_row and abs(start_col - target_col) == 1:
                moves.append((target_row + direction, target_col))

        return moves

    def promote(self, board, row, col):
        """Превращает пешку в другую фигуру."""
        if self.color == 'white' and row == 0:
            self.can_promote = True
        elif self.color == 'black' and row == 7:
            self.can_promote = True

        if self.can_promote:
            print("Пешка дошла до края, можете её заменить на:")
            if board.chess_variant == "standard":
                print("коня, слона, ладью, ферзя или нет.")
            else:
                print("коня, слона, ладью, ферзя, арбалетчика, мага, охотника или нет.")

            choice = input("Ваш выбор: ").strip().lower()
            if choice == "нет":
                return False  # Превращение отложено
            else:
                # Превращение пешки в выбранную фигуру
                if choice == "конь":
                    new_piece = Knight(self.color)
                elif choice == "слон":
                    new_piece = Bishop(self.color)
                elif choice == "ладья":
                    new_piece = Rook(self.color)
                elif choice == "ферзь":
                    new_piece = Queen(self.color)
                elif board.chess_variant == "custom":
                    if choice == "арбалетчик":
                        new_piece = Crossbowman(self.color)
                    elif choice == "маг":
                        new_piece = Mage(self.color)
                    elif choice == "охотник":
                        new_piece = Hunter(self.color)
                    else:
                        print("Некорректный выбор. Превращение отменено.")
                        return False
                else:
                    print("Некорректный выбор. Превращение отменено.")
                    return False

                # Заменяем пешку на новую фигуру
                board.board[row][col] = new_piece
                self.can_promote = False  # Сбрасываем флаг превращения
                return True
        return False

    def move(self, board, start_row, start_col, end_row, end_col):
        # Проверка на взятие на проходе
        if self.en_passant_target:
            target_row, target_col = self.en_passant_target
            if end_row == target_row + (-1 if self.color == 'white' else 1) and end_col == target_col:
                # Удаляем пешку противника
                board.board[target_row][target_col] = None

        # Обновляем флаг первого хода
        if not self.has_moved:
            self.has_moved = True

        # Обновляем позицию пешки
        board.board[end_row][end_col] = self
        board.board[start_row][start_col] = None

        # Сбрасываем возможность взятия на проходе
        self.en_passant_target = None


class Rook(Piece):
    """Класс для ладьи."""

    def __init__(self, color):
        super().__init__(color, '♖' if color == 'white' else '♜')

    def possible_moves(self, board, start_row, start_col):
        moves = []

        # Вверх
        for row in range(start_row - 1, -1, -1):
            if board.board[row][start_col] is None:
                moves.append((row, start_col))
            else:
                if board.board[row][start_col].color != self.color:
                    moves.append((row, start_col))
                break
        # Вниз
        for row in range(start_row + 1, 8):
            if board.board[row][start_col] is None:
                moves.append((row, start_col))
            else:
                if board.board[row][start_col].color != self.color:
                    moves.append((row, start_col))
                break

        # Влево
        for col in range(start_col - 1, -1, -1):
            if board.board[start_row][col] is None:
                moves.append((start_row, col))
            else:
                if board.board[start_row][col].color != self.color:
                    moves.append((start_row, col))
                break

        # Вправо
        for col in range(start_col + 1, 8):
            if board.board[start_row][col] is None:
                moves.append((start_row, col))
            else:
                if board.board[start_row][col].color != self.color:
                    moves.append((start_row, col))
                break
        return moves


class Knight(Piece):
    """Класс для коня."""

    def __init__(self, color):
        super().__init__(color, '♘' if color == 'white' else '♞')

    def possible_moves(self, board, start_row, start_col):
        moves = []
        possible_moves = [
            (start_row - 2, start_col - 1), (start_row - 2, start_col + 1),
            (start_row - 1, start_col - 2), (start_row - 1, start_col + 2),
            (start_row + 1, start_col - 2), (start_row + 1, start_col + 2),
            (start_row + 2, start_col - 1), (start_row + 2, start_col + 1)
        ]
        for row, col in possible_moves:
            if 0 <= row < 8 and 0 <= col < 8:
                if board.board[row][col] is None or board.board[row][col].color != self.color:
                    moves.append((row, col))
        return moves


class Bishop(Piece):
    """Класс для слона."""

    def __init__(self, color):
        super().__init__(color, '♗' if color == 'white' else '♝')

    def possible_moves(self, board, start_row, start_col):
        moves = []
        # Диагональ вверх-влево
        row, col = start_row - 1, start_col - 1
        while 0 <= row < 8 and 0 <= col < 8:
            if board.board[row][col] is None:
                moves.append((row, col))
            else:
                if board.board[row][col].color != self.color:
                    moves.append((row, col))
                break
            row -= 1
            col -= 1

        # Диагональ вверх-вправо
        row, col = start_row - 1, start_col + 1
        while 0 <= row < 8 and 0 <= col < 8:
            if board.board[row][col] is None:
                moves.append((row, col))
            else:
                if board.board[row][col].color != self.color:
                    moves.append((row, col))
                break
            row -= 1
            col += 1

        # Диагональ вниз-влево
        row, col = start_row + 1, start_col - 1
        while 0 <= row < 8 and 0 <= col < 8:
            if board.board[row][col] is None:
                moves.append((row, col))
            else:
                if board.board[row][col].color != self.color:
                    moves.append((row, col))
                break
            row += 1
            col -= 1

        # Диагональ вниз-вправо
        row, col = start_row + 1, start_col + 1
        while 0 <= row < 8 and 0 <= col < 8:
            if board.board[row][col] is None:
                moves.append((row, col))
            else:
                if board.board[row][col].color != self.color:
                    moves.append((row, col))
                break
            row += 1
            col += 1
        return moves


class Queen(Piece):
    """Класс для ферзя."""

    def __init__(self, color):
        super().__init__(color, '♕' if color == 'white' else '♛')

    def possible_moves(self, board, start_row, start_col):
        # Ферзь - это комбинация ладьи и слона
        rook_moves = Rook(self.color).possible_moves(board, start_row, start_col)
        bishop_moves = Bishop(self.color).possible_moves(board, start_row, start_col)
        return rook_moves + bishop_moves


class King(Piece):
    """Класс для короля."""

    def __init__(self, color):
        super().__init__(color, '♔' if color == 'white' else '♚')

    def possible_moves(self, board, start_row, start_col):
        moves = []
        possible_moves = [
            (start_row - 1, start_col - 1), (start_row - 1, start_col), (start_row - 1, start_col + 1),
            (start_row, start_col - 1), (start_row, start_col + 1),
            (start_row + 1, start_col - 1), (start_row + 1, start_col), (start_row + 1, start_col + 1)
        ]
        for row, col in possible_moves:
            if 0 <= row < 8 and 0 <= col < 8:
                if board.board[row][col] is None or board.board[row][col].color != self.color:
                    moves.append((row, col))
        return moves


class Crossbowman(Piece):
    """Класс для арбалетчика."""

    def __init__(self, color):
        super().__init__(color, 'C' if color == 'white' else 'c')

    def possible_moves(self, board, start_row, start_col):
        moves = []
        # Движение на одну клетку в любом направлении
        directions = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1),          (0, 1),
                      (1, -1),  (1, 0), (1, 1)]
        for dr, dc in directions:
            new_row, new_col = start_row + dr, start_col + dc
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board.board[new_row][new_col] is None or board.board[new_row][new_col].color != self.color:
                    moves.append((new_row, new_col))

        # Атака по прямой на расстояние до трех клеток
        attack_directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dr, dc in attack_directions:
            for i in range(1, 4):
                new_row, new_col = start_row + dr * i, start_col + dc * i
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if board.board[new_row][new_col] is not None:
                        if board.board[new_row][new_col].color != self.color:
                            moves.append((new_row, new_col))
                        break
        return moves


class Mage(Piece):
    """Класс для мага."""

    def __init__(self, color):
        super().__init__(color, 'M' if color == 'white' else 'm')

    def possible_moves(self, board, start_row, start_col):
        moves = []
        # Все возможные направления: горизонталь, вертикаль, диагонали
        directions = [
            (-2, -2), (-2, 0), (-2, 2),
            (0, -2),          (0, 2),
            (2, -2),  (2, 0), (2, 2)
        ]

        for dr, dc in directions:
            new_row, new_col = start_row + dr, start_col + dc
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                target_piece = board.board[new_row][new_col]
                if target_piece is None or target_piece.color != self.color:
                    moves.append((new_row, new_col))

        return moves


class Hunter(Piece):
    """Класс для охотника."""

    def __init__(self, color):
        super().__init__(color, 'H' if color == 'white' else 'h')

    def possible_moves(self, board, start_row, start_col):
        moves = []
        # Движение как конь
        knight_moves = [
            (start_row - 2, start_col - 1), (start_row - 2, start_col + 1),
            (start_row - 1, start_col - 2), (start_row - 1, start_col + 2),
            (start_row + 1, start_col - 2), (start_row + 1, start_col + 2),
            (start_row + 2, start_col - 1), (start_row + 2, start_col + 1)
        ]
        for row, col in knight_moves:
            if 0 <= row < 8 and 0 <= col < 8:
                if board.board[row][col] is None or board.board[row][col].color != self.color:
                    moves.append((row, col))

        # Атака на расстоянии двух клеток по диагонали
        diagonal_moves = [
            (start_row - 2, start_col - 2), (start_row - 2, start_col + 2),
            (start_row + 2, start_col - 2), (start_row + 2, start_col + 2)
        ]
        for row, col in diagonal_moves:
            if 0 <= row < 8 and 0 <= col < 8:
                if board.board[row][col] is not None and board.board[row][col].color != self.color:
                    moves.append((row, col))
        return moves


class Checker(Piece):
    """Класс для шашки."""

    def __init__(self, color, is_king=False):
        symbol = 'W' if color == 'white' else 'B'
        if is_king:
            symbol = symbol.lower()  # Король отличается регистром
        super().__init__(color, symbol)
        self.is_king = is_king
        self.has_captured = False  # Флаг для отслеживания съедения фигуры в текущем ходе

    def possible_moves(self, board, start_row, start_col):
        moves = []
        captures = []  # Отдельный список для захватов, чтобы приоритезировать их
        directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]  # Все направления

        if not self.is_king:
            # Обычные шашки могут ходить только в определенном направлении
            if self.color == 'white':
                directions = [(-1, -1), (-1, 1)]  # только вверх для белых
            else:
                directions = [(1, -1), (1, 1)]  # только вниз для черных

        for dr, dc in directions:
            new_row, new_col = start_row + dr, start_col + dc

            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board.board[new_row][new_col] is None:  # простой ход
                    moves.append((new_row, new_col))
                else:  # попытка захвата
                    jump_row, jump_col = new_row + dr, new_col + dc
                    if 0 <= jump_row < 8 and 0 <= jump_col < 8 and board.board[jump_row][jump_col] is None:
                        opponent = board.board[new_row][new_col]
                        if opponent.color != self.color:
                            captures.append((jump_row, jump_col, new_row, new_col))  # добавляем захват и позицию захваченной шашки

        # Если есть захваты, то можно ходить только захватом
        if captures:
            return captures
        else:
            return moves

    def can_continue_capturing(self, board, row, col):
        """Проверяет, может ли шашка продолжать захватывать фигуры."""
        captures = self.possible_moves(board, row, col)
        return any(len(move) == 4 for move in captures)

    def continue_capturing(self, board, row, col):
        """Продолжает захватывать фигуры, если это возможно."""
        while self.can_continue_capturing(board, row, col):
            board.display_board()
            print(f"Вы можете продолжить захват фигур с позиции {chr(ord('a') + col)}{8 - row}.")
            print("Введите целевую позицию для продолжения захвата (например, 'a4') или 'завершить ход' для завершения.")
            command = input("Ваш выбор: ").strip().lower()
            if command == "завершить ход":
                break
            try:
                end_row, end_col = get_coordinates(command)
                if self.is_valid_move(board, row, col, end_row, end_col):
                    self.move(board, row, col, end_row, end_col)
                    row, col = end_row, end_col
                else:
                    print("Недопустимый ход.")
                    break
            except (ValueError, IndexError):
                print("Некорректный ввод. Пожалуйста, введите позицию в формате 'a2'.")
                break

    def move(self, board, start_row, start_col, end_row, end_col):
        """Перемещает шашку и проверяет, нужно ли превращение в короля."""
        if self.is_king_move(start_row, end_row):
            self.promote_to_king()
        board.board[end_row][end_col] = self
        board.board[start_row][start_col] = None

    def is_king_move(self, start_row, end_row):
        """Проверяет, становится ли шашка королем после хода."""
        if self.color == 'white' and end_row == 0:
            return True
        elif self.color == 'black' and end_row == 7:
            return True
        return False

    def promote_to_king(self):
        """Превращает шашку в короля."""
        self.is_king = True
        self.symbol = self.symbol.lower()


class Board:
    """Класс, представляющий игровую доску (шахматы или шашки)."""

    def __init__(self, game_type="chess", chess_variant="standard"):
        self.board = [[None for _ in range(8)] for _ in range(8)]
        self.game_type = game_type
        self.chess_variant = chess_variant
        self.move_history = []  # История ходов
        self.en_passant_target = None  # Поле для отслеживания взятия на проходе
        self.setup_board()

    def setup_board(self):
        """Устанавливает начальную расстановку фигур на доске в зависимости от типа игры."""
        if self.game_type == "checkers":
            self.setup_checkers()
        elif self.game_type == "chess":
            if self.chess_variant == "standard":
                self.setup_standard_chess()
            elif self.chess_variant == "custom":
                self.setup_custom_chess()

    def setup_checkers(self):
        """Устанавливает начальную расстановку шашек."""
        for row in range(3):
            for col in range(8):
                if (row + col) % 2 == 1:  # Шашки ставятся только на черные клетки
                    self.board[row][col] = Checker('black')

        for row in range(5, 8):
            for col in range(8):
                if (row + col) % 2 == 1:
                    self.board[row][col] = Checker('white')

    def setup_standard_chess(self):
        """Устанавливает начальную расстановку стандартных шахмат."""
        # Белые
        self.board[7][0] = Rook('white')
        self.board[7][1] = Knight('white')
        self.board[7][2] = Bishop('white')
        self.board[7][3] = Queen('white')
        self.board[7][4] = King('white')
        self.board[7][5] = Bishop('white')
        self.board[7][6] = Knight('white')
        self.board[7][7] = Rook('white')
        for i in range(8):
            self.board[6][i] = Pawn('white')

        # Черные
        self.board[0][0] = Rook('black')
        self.board[0][1] = Knight('black')
        self.board[0][2] = Bishop('black')
        self.board[0][3] = Queen('black')
        self.board[0][4] = King('black')
        self.board[0][5] = Bishop('black')
        self.board[0][6] = Knight('black')
        self.board[0][7] = Rook('black')
        for i in range(8):
            self.board[1][i] = Pawn('black')

    def setup_custom_chess(self):
        """Устанавливает начальную расстановку шахмат с дополнительными фигурами."""
        # Белые
        self.board[7][0] = Rook('white')
        self.board[7][1] = Knight('white')
        self.board[7][2] = Crossbowman('white')
        self.board[7][3] = Queen('white')
        self.board[7][4] = King('white')
        self.board[7][5] = Mage('white')
        self.board[7][6] = Hunter('white')
        self.board[7][7] = Rook('white')
        for i in range(8):
            self.board[6][i] = Pawn('white')

        # Черные
        self.board[0][0] = Rook('black')
        self.board[0][1] = Knight('black')
        self.board[0][2] = Crossbowman('black')
        self.board[0][3] = Queen('black')
        self.board[0][4] = King('black')
        self.board[0][5] = Mage('black')
        self.board[0][6] = Hunter('black')
        self.board[0][7] = Rook('black')
        for i in range(8):
            self.board[1][i] = Pawn('black')

    def display_board(self):
        """Выводит игровую доску в консоль."""
        print("  a b c d e f g h")
        for row in range(8):
            print(8 - row, end=" ")
            for col in range(8):
                piece = self.board[row][col]
                if piece:
                    print(piece, end=" ")
                else:
                    print(". ", end="")
            print(8 - row)
        print("  a b c d e f g h")

    def is_king_under_attack(self, color):
        """Проверяет, находится ли король под шахом."""
        king_pos = None
        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if isinstance(piece, King) and piece.color == color:
                    king_pos = (row, col)
                    break
            if king_pos:
                break

        if not king_pos:
            return False

        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece and piece.color != color:
                    if piece.is_valid_move(self, row, col, king_pos[0], king_pos[1]):
                        return True
        return False

    def move_piece(self, start_row, start_col, end_row, end_col):
        """Перемещает фигуру с одной позиции на другую."""
        piece = self.board[start_row][start_col]
        if piece is None:
            print("Нет фигуры для перемещения в этой позиции.")
            return False

        if not piece.is_valid_move(self, start_row, start_col, end_row, end_col):
            print("Недопустимый ход для этой фигуры.")
            return False

        # Сохраняем ход в истории
        self.move_history.append(((start_row, start_col), (end_row, end_col), self.board[end_row][end_col]))

        # Проверка на съедение фигуры
        captured_piece = self.board[end_row][end_col]
        if captured_piece is not None:
            piece.has_captured = True  # Устанавливаем флаг, что фигура была съедена

            # Проверка, была ли съедена фигура короля
            if isinstance(captured_piece, King):
                print(f"Игра окончена: победили {'Белые' if piece.color == 'white' else 'Черные'}!")
                return False  # Завершаем игру

        # Взятие на проходе
        if isinstance(piece, Pawn) and self.en_passant_target:
            target_row, target_col = self.en_passant_target
            if end_row == target_row + (-1 if piece.color == 'white' else 1) and end_col == target_col:
                # Удаляем пешку противника
                self.board[target_row][target_col] = None

        self.board[end_row][end_col] = piece
        self.board[start_row][start_col] = None

        # Обновляем возможность взятия на проходе
        if isinstance(piece, Pawn) and abs(start_row - end_row) == 2:
            self.en_passant_target = (end_row, end_col)
        else:
            self.en_passant_target = None

        # Проверка на превращение пешки
        if isinstance(piece, Pawn):
            if (piece.color == 'white' and end_row == 0) or (piece.color == 'black' and end_row == 7):
                if not piece.promote(self, end_row, end_col):
                    # Если пользователь выбрал "нет", оставляем пешку
                    print("Превращение отложено. Вы сможете превратить пешку в следующий раз.")
                else:
                    print("Пешка успешно превращена.")

        # Проверка на превращение пешки в шашках
        if self.game_type == "checkers":
            if isinstance(piece, Checker):
                if piece.color == 'white' and end_row == 0:
                    piece = Checker(piece.color, is_king=True)
                    self.board[end_row][end_col] = piece
                elif piece.color == 'black' and end_row == 7:
                    piece = Checker(piece.color, is_king=True)
                    self.board[end_row][end_col] = piece

                # Если была съедена фигура и есть возможность продолжить захват, предлагаем это сделать
                if piece.has_captured and piece.can_continue_capturing(self, end_row, end_col):
                    piece.continue_capturing(self, end_row, end_col)

        # Проверка на шах в шахматах
        if self.game_type == "chess":
            if self.is_king_under_attack('white' if piece.color == 'black' else 'black'):
                print("Вам шах!")

        return True

    def undo_move(self):
        """Откатывает последний ход."""
        if not self.move_history:
            print("Нет ходов для отката.")
            return False

        # Восстанавливаем последний ход
        (start_row, start_col), (end_row, end_col), captured_piece = self.move_history.pop()
        self.board[start_row][start_col] = self.board[end_row][end_col]
        self.board[end_row][end_col] = captured_piece
        return True

    def show_possible_moves(self, row, col):
        """Показывает возможные ходы для выбранной фигуры."""
        piece = self.board[row][col]
        if piece is None:
            print("В данной клетке нет фигуры.")
            return

        moves = piece.possible_moves(self, row, col)
        print(f"Доступные ходы для фигуры {piece}:")
        for move in moves:
            print(f"{chr(ord('a') + move[1])}{8 - move[0]}", end=" ")
        print()

    def show_threatened_pieces(self, color):
        """Показывает фигуры, находящиеся под боем."""
        threatened_pieces = []
        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece is not None and piece.color == color:
                    for enemy_row in range(8):
                        for enemy_col in range(8):
                            enemy_piece = self.board[enemy_row][enemy_col]
                            if enemy_piece is not None and enemy_piece.color != color:
                                if enemy_piece.is_valid_move(self, enemy_row, enemy_col, row, col):
                                    threatened_pieces.append((row, col))
                                    break
        if threatened_pieces:
            print(f"Фигуры {color} под боем:")
            for piece in threatened_pieces:
                print(f"{chr(ord('a') + piece[1])}{8 - piece[0]}", end=" ")
            print()
        else:
            print(f"Нет фигур {color} под боем.")

    def get_piece(self, row, col):
        """Возвращает фигуру на заданной позиции."""
        if 0 <= row < 8 and 0 <= col < 8:
            return self.board[row][col]
        return None


def get_coordinates(position):
    """Преобразует шахматную позицию (например, 'a2') в координаты (строка, столбец)."""
    col = ord(position[0]) - ord('a')
    row = 8 - int(position[1])
    return row, col


def main():
    """Основная функция для запуска игры."""
    game_type = input("Выберите игру (chess или checkers): ").lower()
    while game_type not in ("chess", "checkers"):
        game_type = input("Некорректный ввод. Выберите игру (chess или checkers): ").lower()

    chess_variant = "standard"
    if game_type == "chess":
        chess_variant = input("Выберите вариант шахмат (standard или custom): ").lower()
        while chess_variant not in ("standard", "custom"):
            chess_variant = input("Некорректный ввод. Выберите вариант шахмат (standard или custom): ").lower()

    board = Board(game_type, chess_variant)
    board.display_board()
    move_count = 0
    current_player = 'white'

    while True:
        print(f"Ход {move_count + 1}. Ход {current_player}.")

        # Проверка на шах перед ходом
        if game_type == "chess":
            if board.is_king_under_attack(current_player):
                print("Вам шах! Ваш король находится под угрозой.")

        command = input("Введите команду (ход/откат/подсказка/угрозы/+ для выхода): ").strip().lower()

        # Остановка программы при вводе "+"
        if command == "+":
            print("Игра завершена.")
            break

        if command == "откат":
            if board.undo_move():
                board.display_board()
                move_count -= 1
                current_player = 'black' if current_player == 'white' else 'white'
            continue

        if command == "подсказка":
            position = input("Введите позицию фигуры для подсказки (например, 'a2'): ")
            try:
                row, col = get_coordinates(position)
                board.show_possible_moves(row, col)
            except (ValueError, IndexError):
                print("Некорректный ввод. Пожалуйста, введите позицию в формате 'a2'.")
            continue

        if command == "угрозы":
            board.show_threatened_pieces(current_player)
            continue

        start_position = input("Введите позицию фигуры для перемещения (например, 'a2'): ")
        end_position = input("Введите целевую позицию (например, 'a4'): ")

        try:
            start_row, start_col = get_coordinates(start_position)
            end_row, end_col = get_coordinates(end_position)

            piece = board.get_piece(start_row, start_col)
            if piece is None:
                print("В данной клетке нет фигуры.")
                continue
            if piece.color != current_player:
                print(f"Сейчас ход {current_player}.")
                continue

            # Если ход успешен, проверяем, не был ли съеден король
            if board.move_piece(start_row, start_col, end_row, end_col):
                board.display_board()
                move_count += 1
                current_player = 'black' if current_player == 'white' else 'white'
            else:
                # Если метод move_piece вернул False, значит, король был съеден, и игра завершена
                if board.get_piece(end_row, end_col) and isinstance(board.get_piece(end_row, end_col), King):
                    print("Игра завершена.")
                    break  # Выход из цикла while True
                else:
                    print("Недопустимый ход. Попробуйте снова.")
                    continue  # Продолжаем игру

        except (ValueError, IndexError):
            print("Некорректный ввод. Пожалуйста, введите позицию в формате 'a2'.")
        except Exception as e:
            print(f"Произошла ошибка: {e}")

# Запуск игры
if __name__ == "__main__":

    main()

Выберите игру (chess или checkers): checkers
  a b c d e f g h
8 . B . B . B . B 8
7 B . B . B . B . 7
6 . B . B . B . B 6
5 . . . . . . . . 5
4 . . . . . . . . 4
3 W . W . W . W . 3
2 . W . W . W . W 2
1 W . W . W . W . 1
  a b c d e f g h
Ход 1. Ход white.
Введите команду (ход/откат/подсказка/угрозы/+ для выхода): a3
Введите позицию фигуры для перемещения (например, 'a2'): a3
Введите целевую позицию (например, 'a4'): b4
Недопустимый ход. Попробуйте снова.
Ход 1. Ход white.
Введите команду (ход/откат/подсказка/угрозы/+ для выхода): ход
Введите позицию фигуры для перемещения (например, 'a2'): a3
Введите целевую позицию (например, 'a4'): b4
В данной клетке нет фигуры.
Ход 1. Ход white.


KeyboardInterrupt: Interrupted by user